In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import random

In [ ]:
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

In [2]:
#subtract by the mean and divide by the standard deviation
#normalize data
# X_train = (X_train - np.mean(X_train)) / np.std(X_train)

#unused method to scale from 0 to 1~ish
def preproc(unclean_batch_x):
    """Convert values to range 0-1"""
    temp_batch = unclean_batch_x / unclean_batch_x.max()
    return temp_batch
#X_train = preproc(X_train)

def shuffle(Xtrain, ytrain):
    stacked = np.column_stack((Xtrain,ytrain))
    np.random.shuffle(stacked)
    return stacked[:,:Xtrain.shape[1]], stacked[:,Xtrain.shape[1]]

def get_kernel_size(factor):
    """
    Find the kernel size given the desired factor of upsampling.
    """
    return 2 * factor - factor % 2


def upsample_filt(size):
    """
    Make a 2D bilinear kernel suitable for upsampling of the given (h, w) size.
    """
    factor = (size + 1) // 2
    if size % 2 == 1:
        center = factor - 1
    else:
        center = factor - 0.5
    og = np.ogrid[:size, :size]
    return (1 - abs(og[0] - center) / factor) * \
           (1 - abs(og[1] - center) / factor)


def bilinear_upsample_weights(factor, number_of_classes):
    """
    Create weights matrix for transposed convolution with bilinear filter
    initialization.
    """
    filter_size = get_kernel_size(factor)
    weights = np.zeros((filter_size,
                        filter_size,
                        number_of_classes,
                        number_of_classes), dtype=np.float32)
    
    upsample_kernel = upsample_filt(filter_size)
    
    for i in xrange(number_of_classes):
        weights[:, :, i, i] = upsample_kernel
    return weights

def upsample_bilinear_interpolation(conv, scale):
    old_height = conv.shape()[0]
    old_width = conv.shape()[1]
    new_height = int(round(old_height * scale))
    new_width = int(round(old_width * scale))
    resized = tf.image.resize_images(input_tensor, [new_height, new_width])
    return resized

def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

In [ ]:
class IntrinsicNetwork(object):
    def __init__(self, image):
    self.image = image
    
    # wrapper for applying spatial conv, batchnorm, reLU
    def conv2d(x, W, b, strides=1):
        # Conv2D wrapper, with bias and relu activation
        x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
        x = tf.nn.bias_add(x, b)
        # Batch Norm
        x = tf.contrib.layers.batch_norm(x, center=True, scale=True, is_training=phase, scope='bn')
        return tf.nn.relu(x)
    
    def deconv2d():
        return None
    
    # Create model
    def encoder_layers(x, weights, biases, dropout, dropout2):
        # Reshape input picture
        x = tf.reshape(x, shape=[-1, 28, 28, 1])

        # Convolution Layer + relu
        conv0 = conv2d(x, weights['wcconv0'], biases['bc0'])
        print("conv0 and relu", conv0.shape)

        # Convolution Layer + relu
        conv1 = conv2d(x, weights['wcconv1'], biases['bc1'])
        print("conv1 and relu", conv1.shape)

        # Convolution Layer + relu
        conv2 = conv2d(conv1, weights['wcconv2'], biases['bc2'])
        print("conv2 and relu", conv2.shape)

        # Convolution Layer + relu
        conv3 = conv2d(conv2, weights['wcconv3'], biases['bc3'])
        print("conv3 and relu", conv3.shape)

        # Convolution Layer + relu
        conv4 = conv2d(conv3, weights['wcconv4'], biases['bc4'])
        print("conv4 and relu", conv4.shape)

        # Convolution Layer + relu
        conv5 = conv2d(conv4, weights['wcconv5'], biases['bc4'])
        print("conv5 and relu", conv5.shape)

        return [conv0, conv1, conv2, conv3, conv4, conv5]
    
    def decoder_layers(convs, rep_weights, rep_biases, axis=0):
        
        mid = conv2d(conv2d(conv2d(conv2d(conv[5], weights['fc'], biases['fc']),
                                          , weights['fc'], biases['fc']),
                            , weights['fc'], biases['fc']),
                     , weights['fc'], biases['fc'])
        mid_1 = convs[5]
        
        dec0 = tf.concat([mid, mid_1], axis)
        deconv0 = upsample_tf(factor=2, input_img=conv2d(dec0, weights['wdconv0'], biases['bd0']))
        deconv0_1 = convs[4]
        
        dec1 = tf.concat([deconv0, deconv0_1], axis)
        deconv1 = upsample_tf(factor=2, input_img=conv2d(dec1, weights['wdconv1'], biases['bd1']))
        deconv1_1 = convs[3]
        
        dec2 = tf.concat([deconv1, deconv1_1], axis)
        deconv2 = upsample_tf(factor=2, input_img=conv2d(dec2, weights['wdconv2'], biases['bd2']))
        deconv2_1 = convs[2]
        
        dec3 = tf.concat([deconv2, deconv2_1], axis)
        deconv3 = upsample_tf(factor=2, input_img=conv2d(dec3, weights['wdconv3'], biases['bd3']))
        deconv3_1 = convs[1]
        
        dec4 = tf.concat([deconv3, deconv3_1], axis)
        deconv4 = upsample_tf(factor=2, input_img=conv2d(dec4, weights['wdconv4'], biases['bd4']))
        deconv4_1 = convs[0]
        
        dec_out = tf.concat([deconv4, deconv4_1], axis)
        output = conv2d(dec_out, weights['output'], biases['out'])
        res = conv2d(output, weights['res'], biases['res'])
        
        return [deconv0, deconv1, deconv2, deconv3, deconv4], res
        

In [ ]:
# Store layers weight & bias
# more parameters
filter_size = 3

wc0_depth = 16
wc1_depth = 32
wc2_depth = 64
wc3_depth = 128
wc4_depth = 256
wc5_depth = 256

fc1_depth = 256

weights = {
    'wcconv0': tf.Variable(tf.random_normal([filter_size, filter_size, 3, wc0_depth])), #
    'wcconv1': tf.Variable(tf.random_normal([filter_size, filter_size, wc0_depth, wc1_depth])), #
    'wcconv2': tf.Variable(tf.random_normal([filter_size, filter_size, wc1_depth, wc2_depth])), #
    'wcconv3': tf.Variable(tf.random_normal([filter_size, filter_size, wc2_depth, wc3_depth])), #
    'wcconv4': tf.Variable(tf.random_normal([filter_size, filter_size, wc3_depth, wc4_depth])), #
    'wcconv5': tf.Variable(tf.random_normal([filter_size, filter_size, wc4_depth, wc5_depth])), #
    
    'fc': tf.Variable(tf.random_normal([filter_size, filter_size, fc1_depth, fc1_depth])),
    'wdconv0': tf.Variable(tf.random_normal([filter_size, filter_size, 1024, 256])),
    'wdconv1': tf.Variable(tf.random_normal([filter_size, filter_size, 1024, 128])),
    'wdconv2': tf.Variable(tf.random_normal([filter_size, filter_size, 512, 64])),
    'wdconv3': tf.Variable(tf.random_normal([filter_size, filter_size, 256, 32])),
    'wdconv4': tf.Variable(tf.random_normal([filter_size, filter_size, 128, 16])),
    'output': tf.Variable(tf.random_normal([filter_size, filter_size, 64, 16])),
    'res': tf.Variable(tf.random_normal([filter_size, filter_size, wc0_depth, 3])),
    # fully connected
    'wd1': tf.Variable(tf.random_normal([7*7*wc2_depth, fc1_depth])), #
    'out': tf.Variable(tf.random_normal([fc1_depth, output_size])) #
}

biases = {
    'bc0': tf.Variable(tf.random_normal([wc0_depth])),
    'bc1': tf.Variable(tf.random_normal([wc1_depth])),
    'bc2': tf.Variable(tf.random_normal([wc2_depth])),
    'bc3': tf.Variable(tf.random_normal([wc3_depth])),
    'bc4': tf.Variable(tf.random_normal([wc4_depth])),
    'bc5': tf.Variable(tf.random_normal([wc5_depth])),
    
    'fc': tf.Variable(tf.random_normal([fc1_depth])),
    'bd0': tf.Variable(tf.random_normal([1024])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'bd2': tf.Variable(tf.random_normal([512])),
    'bd3': tf.Variable(tf.random_normal([256])),
    'bd4': tf.Variable(tf.random_normal([128])),
    'out': tf.Variable(tf.random_normal([64])),
    'res': tf.Variable(tf.random_normal([output_size]))
}


mid_w = tf.Variable(tf.random_normal([filter_size, filter_size, 3, wc0_depth])), #
mid_b = tf.Variable(tf.random_normal([filter_size, filter_size, 3, wc0_depth])), #
    
# Construct model
convs = encoder_layers(x, weights, biases, keep_prob, keep_prob2)
pred = decoder_layer(convs, rep_weights, rep_biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
# optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In [ ]:
# batch size of one = stochastic gradient descent, mini-batch gradient descent
# should use 10 at once
# mini batch neural nets tensorflow
# fc, relu, maybe a few repetitions, softmax

M = 70 #M is the input size factor

input_size = 32*M # 28 x 28
output_size = 32*M

# dropout = 0.8 # probability of keeping that data point
# dropout2 = 0.9

# learning_rate = 0.5

# data placeholders

# input data
x = tf.placeholder(tf.float32, shape=[None, input_size])
# correct answers
y = tf.placeholder(tf.float32, [None, output_size])
# dropout variable
keep_prob = tf.placeholder(tf.float32)
keep_prob2 = tf.placeholder(tf.float32)

In [ ]:
# more parameters
batch_size = 30
epochs = 5

display_step = 1

n_samples = x_training.shape[0]
#rounds_number = data_size/ batch_size

mean_losses = []
mean_accs = []

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    # Keep training until reach max iterations
    for epoch in range(epochs):
        print("running epoch: ", epoch)
        total_iter = n_samples // batch_size #total_iter
        total_loss = 0
#         indeces = np.random.permutations(np.arange(len(n_samples))
        
        for i in range(total_iter):
#             x = X[i * batch_size : (i + 1) * batch_size]
            start = i * batch_size
            end = (i + 1) * batch_size
            batch_x, batch_y = x_training[start:end], labels_training[start:end]
            
            # Run optimization op (backprop)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout, keep_prob2: dropout2})
            
            # Calculate batch loss and accuracy 
#             loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 1., keep_prob2: 1.})
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x, y: batch_y, keep_prob: 1., keep_prob2: 1.})
#             loss = self.fit_batch(x)
            total_loss += loss
#             total_acc += acc

            if i%100 == 0:
                print("epoch: ",epoch,", iteration: ",i,", loss: ",loss," total loss: ",total_loss)
    
    
        mean_loss = total_loss / total_iter
        mean_losses.append(mean_loss)
        
#         mean_acc = total_acc / total_iter
#         mean_accs.append(mean_acc)
        
        if (epoch + 1) % display_step == 0:
            print('epoch %s: loss=%.4f' % (epoch + 1, mean_loss))
            
    
    # Calculate accuracy for test images
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: x_valid, y: labels_valid, keep_prob: 1., keep_prob2: 1.}))

In [ ]:
# Unused implementation of fully connected layer

# Fully connected layer
# Reshape conv2 output to fit fully connected layer input
fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
print("fc1 shape", fc1.shape)
fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
fc1 = tf.nn.relu(fc1)
print("fc1 shape after weights and biases", fc1.shape)
# Apply Dropout
fc1 = tf.nn.dropout(fc1, dropout)

# Output, class prediction (dense layer)
fc2 = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
print("fc2 shape after weights and biases", fc2.shape)
#     fc2 = tf.nn.dropout(fc2, dropout2)